In [6]:
import numpy as np
import math

np.random.seed(42)  # reproducible

# -------- Problem Parameters --------
n_periods = 10                      # Number of time periods
max_capacity = 1000                 # Max reservoir capacity (units)
# Use fixed inflow (printable) for reproducibility; you can replace with random if desired
inflow = np.random.randint(50, 150, size=n_periods)
initial_storage = 0                 # start reservoir volume

# -------- Cuckoo Search Parameters --------
n_nests = 20
pa = 0.25                           # discovery rate
max_iter = 100
step_size = 0.1

print("Inflow per period:", inflow)

# -------- Utility: repair schedule to ensure realism and compute penalties --------
def evaluate_schedule(schedule):
    """
    Simulate the schedule forward, repair any release > available volume,
    compute total_released and penalty for spills or attempted over-releases.
    Returns: fitness_value, total_released, penalty
    """
    volume = initial_storage
    total_released = 0.0
    penalty = 0.0

    for t in range(n_periods):
        # Add inflow for this period
        volume += inflow[t]

        # Proposed release
        release = float(schedule[t])

        # If release is negative, penalize and set to 0
        if release < 0:
            penalty += 100.0 * abs(release)  # small penalty for negative proposal
            release = 0.0

        # If release exceeds available water, penalize the attempted overshoot and clamp
        if release > volume:
            penalty += 100.0 * (release - volume)  # penalty factor can be tuned
            release = volume

        volume -= release  # actual release

        # If reservoir overflows capacity, this is a spill -> penalize spill
        if volume > max_capacity:
            spill = volume - max_capacity
            penalty += 50.0 * spill   # spill penalty (smaller than overshoot)
            volume = max_capacity

        total_released += release

    fitness_value = total_released - penalty
    return fitness_value, total_released, penalty

# wrapper to match earlier API
def fitness(schedule):
    f, _, _ = evaluate_schedule(schedule)
    return f

# -------- Levy flight generator --------
def levy_flight(Lambda=1.5):
    sigma = (math.gamma(1+Lambda)*np.sin(np.pi*Lambda/2) /
             (math.gamma((1+Lambda)/2)*Lambda*2**((Lambda-1)/2)))**(1/Lambda)
    u = np.random.normal(0, sigma, n_periods)
    v = np.random.normal(0, 1, n_periods)
    step = u / (np.abs(v)**(1.0/Lambda))
    return step

# -------- Initialize nests in a realistic range --------
# Upper bound per period: it's reasonable to start with releases between 0 and mean(inflow)+some buffer
upper_init = max(1.0, np.mean(inflow)*1.5)
nests = np.random.uniform(0, upper_init, (n_nests, n_periods))

# Evaluate initial fitnesses
fitnesses = np.array([fitness(nests[i]) for i in range(n_nests)])
best_idx = np.argmax(fitnesses)
best_nest = nests[best_idx].copy()
best_fitness = fitnesses[best_idx]

# -------- Main Cuckoo Search loop --------
for iteration in range(max_iter):
    for i in range(n_nests):
        # Levy flight step and candidate solution
        step = levy_flight() * step_size
        new_solution = nests[i] + step

        # Quick clipping to a reasonable bound (non-negative and not absurdly large)
        new_solution = np.clip(new_solution, 0.0, max_capacity)

        new_fitness = fitness(new_solution)

        # Replace if better
        if new_fitness > fitnesses[i]:
            nests[i] = new_solution
            fitnesses[i] = new_fitness

        # Update global best
        if new_fitness > best_fitness:
            best_fitness = new_fitness
            best_nest = new_solution.copy()

    # Discovery: replace a fraction 'pa' of nests with new random solutions
    discover_mask = np.random.rand(n_nests) < pa
    # New random nests drawn from realistic range
    random_nests = np.random.uniform(0, upper_init, (n_nests, n_periods))
    # Apply discovery replacement
    nests[discover_mask] = random_nests[discover_mask]

    # Recompute fitnesses and best
    fitnesses = np.array([fitness(nests[i]) for i in range(n_nests)])
    current_best_idx = np.argmax(fitnesses)
    current_best_fitness = fitnesses[current_best_idx]
    if current_best_fitness > best_fitness:
        best_fitness = current_best_fitness
        best_nest = nests[current_best_idx].copy()

    if iteration % 10 == 0 or iteration == max_iter-1:
        bf, tot_rel, pen = evaluate_schedule(best_nest)
        print(f"Iteration {iteration}: Best fitness = {bf:.2f}, total_released = {tot_rel:.2f}, penalty = {pen:.2f}")

# -------- Final (verbose) output --------
final_fitness, final_total_released, final_penalty = evaluate_schedule(best_nest)
print("\nOptimal Release Schedule (repaired/actual feasible values):")
# To show integers (if desired), round to 2 decimals
print(np.round(best_nest, 4))
print(f"Total Released = {final_total_released:.2f}")
print(f"Penalty         = {final_penalty:.2f}")
print(f"Fitness (released - penalty) = {final_fitness:.2f}")


Inflow per period: [101 142  64 121 110  70 132 136 124 124]
Iteration 0: Best fitness = 919.33, total_released = 919.33, penalty = 0.00
Iteration 10: Best fitness = 953.51, total_released = 953.51, penalty = 0.00
Iteration 20: Best fitness = 1066.40, total_released = 1066.40, penalty = 0.00
Iteration 30: Best fitness = 1066.40, total_released = 1066.40, penalty = 0.00
Iteration 40: Best fitness = 1066.40, total_released = 1066.40, penalty = 0.00
Iteration 50: Best fitness = 1066.40, total_released = 1066.40, penalty = 0.00
Iteration 60: Best fitness = 1066.40, total_released = 1066.40, penalty = 0.00
Iteration 70: Best fitness = 1066.40, total_released = 1066.40, penalty = 0.00
Iteration 80: Best fitness = 1066.88, total_released = 1066.88, penalty = 0.00
Iteration 90: Best fitness = 1066.88, total_released = 1066.88, penalty = 0.00
Iteration 99: Best fitness = 1066.88, total_released = 1066.88, penalty = 0.00

Optimal Release Schedule (repaired/actual feasible values):
[ 99.3332  12.